In [ ]:
import pandas as pd
import json 
from sentence_transformers import SentenceTransformer
import glob
import faiss
from fuzzywuzzy import fuzz
from llmsherpa.readers import LayoutPDFReader
import matplotlib.pyplot as plt
import numpy as np
import os

In [ ]:
# !pip install pandas
# !pip install sentence-transformers
# !pip install openpyxl
# !pip install fuzzywuzzy
# !pip install llmsherpa
# !pip install 

## Load model

In [ ]:
model = SentenceTransformer('bert-base-nli-mean-tokens')

In [ ]:
def split_data(data):
    return data["raw_text"].split("\n")

In [ ]:
def read_json_from_folders(directory_path):
    """
    Reads all JSON files from each folder in the specified directory.

    :param directory_path: Path to the directory containing folders of JSON files.
    :return: A list of dictionaries where each dictionary contains data from a single JSON file.
    """
    all_data = []
    filenames = []
    search_pattern = f"{directory_path}/*/*.json"
    for file_path in glob.glob(search_pattern):
        with open(file_path, 'r') as file:
            data = json.load(file)
            data = split_data(data)
            filenames.extend([file_path]*len(data))
            all_data.extend(data)
    return all_data, filenames

In [ ]:
directory_path = r'C:\Users\J C SINGLA\Downloads\External - take_home_challenge_(withJSONs)\take_home_challenge_(withJSONs)\raw_text'
all_data, filenames = read_json_from_folders(directory_path)

In [ ]:
doc_name = "DR--110685614"
subset_str = "C:\\Users\\J C SINGLA\\Downloads\\External - take_home_challenge_(withJSONs)\\take_home_challenge_(withJSONs)\\raw_text\\" +doc_name+"\\raw_text.json"

In [ ]:
sentence_embeddings = model.encode(all_data_sherpa)

In [ ]:
sentence_embeddings.shape

DRAGON+

In [ ]:
import torch
from transformers import AutoTokenizer, AutoModel
tokenizer = AutoTokenizer.from_pretrained('facebook/dragon-plus-query-encoder')
query_encoder = AutoModel.from_pretrained('facebook/dragon-plus-query-encoder')
context_encoder = AutoModel.from_pretrained('facebook/dragon-plus-context-encoder')

# We use msmarco query and passages as an example
query =  "Where was Marie Curie born?"
contexts = [
    "Maria Sklodowska, later known as Marie Curie, was born on November 7, 1867.",
    "Born in Paris on 15 May 1859, Pierre Curie was the son of Eugène Curie, a doctor of French Catholic origin from Alsace."
]
# Apply tokenizer
query_input = tokenizer(query, return_tensors='pt')
ctx_input = tokenizer(contexts, padding=True, truncation=True, return_tensors='pt')
# Compute embeddings: take the last-layer hidden state of the [CLS] token
query_emb = query_encoder(**query_input).last_hidden_state[:, 0, :]
ctx_emb = context_encoder(**ctx_input).last_hidden_state[:, 0, :]
# Compute similarity scores using dot product
score1 = query_emb @ ctx_emb[0]  # 396.5625
score2 = query_emb @ ctx_emb[1]  # 393.8340


In [ ]:
score2

In [ ]:
ctx_emb = context_encoder(**ctx_input).last_hidden_state[:, 0, :]

# Index Flat2

In [ ]:
d = sentence_embeddings.shape[1]
index = faiss.IndexFlatL2(d)
index.add(sentence_embeddings)
index.is_trained

In [ ]:
# # COSINE INDEX
# index = faiss.IndexFlatL2(d)
# sentence_embeddings = sentence_embeddings / np.linalg.norm(sentence_embeddings, axis=1, keepdims=True)
# index.add(sentence_embeddings)
# index.ntotal

# Ground Truth

In [ ]:
ground_truth_path = r"C:\Users\J C SINGLA\Downloads\External - take_home_challenge_(withJSONs)\take_home_challenge_(withJSONs)\document_questions.xlsx"

In [ ]:
ground_truth = pd.read_excel(ground_truth_path)

In [ ]:
# ground_truth

In [ ]:
ground_truth_text = ground_truth[ground_truth["complexity"]=="text"].copy()

In [ ]:
# ground_truth_text

# Inference

In [ ]:
def test_case():
    query = ["What legislative suggestions pertaining to workplace sexual harassment does the firm provide in its submission?", 
                  "What legislative suggestions pertaining to workplace sexual harassment does the firm provide in its submission?"]
    return faiss_inference(query)[0] == faiss_inference(query)[1]

In [ ]:
def faiss_inference(idx, data, query, k=4):
    if isinstance(query, list):
        xq = model.encode(query)
        D, I = idx.search(xq, k)
        retrieved_items = [[data[i] for i in sublist] for sublist in I]
        assert test_case()
    else:
        xq = model.encode([query])
        D, I = idx.search(xq, k)
        retrieved_items = [data[i] for i in list(I[0])]
    return retrieved_items    

In [ ]:
test_data = list(ground_truth_text["relevant questions"])
test_labels = list(ground_truth_text["answer"])

In [ ]:
ret_context = []
for i in range(len(test_data)):
    ret_context.append(faiss_inference(index, all_data_sherpa, test_data[i]))

# Testing 

In [ ]:
def recall_k(test_labels, ret_context, k=len(ret_context[0])):
    ctr = 0
    correct, incorrect = [], []
    for i in range(len(ret_context)):
        done = False
        if isinstance(test_labels[i], float):
            # handle case for when answer is not present in the pdf
            continue
        for j in range(min(k,len(ret_context))):
            if fuzz.partial_ratio(test_labels[i], ret_context[i][j])>=90 and len(ret_context[i][j])>=len(test_labels[i])*0.95:
                # print ("test_labels:", test_labels[i], "\n", "ret_context:", ret_context[i][j])
                ctr += 1
                done = True
                break
                
        if done == False:
            incorrect.append({test_labels[i]:ret_context[i]})

        else:
            correct.append({test_labels[i]:ret_context[i]})

    return ctr/len(ret_context), incorrect, correct

In [ ]:
# all_data_sherpa

In [ ]:
recall, incorrect, correct = recall_k(test_labels, ret_context)

In [ ]:
recall

In [ ]:
incorrect[0].keys()

In [ ]:
list(incorrect[0].values())[0][3]

In [ ]:
len("This year we had 52 students from six universities attending the event. Participating schools were: \nPortland State University \nUniversity of Idaho \nUniversity of Portland \nOregon State University \nUniversity of Washington \nOregon Institute of Technology ")

In [ ]:
# string_A = "This year we had 52 students from six universities attending the event. Participating schools were: \nPortland State University \nUniversity of Idaho \nUniversity of Portland \nOregon State University \nUniversity of Washington \nOregon Institute of Technology"

# string_B = "This year we had 52 students from six universities attending the event. Participating schools were: \nUniversity of Idaho \nUniversity of Portland \nOregon State University \nUniversity of Washington \nOregon Institute of Technology."
string_A = list(incorrect[0].values())[0][0]
string_B = list(incorrect[0].keys())[0]
similarity_score = fuzz.partial_ratio(string_A, string_B)
similarity_score

In [ ]:
# sherpa data
# missing chunking - 3
# not using tokens for search - 2
# too many references to fetch - 1

In [ ]:
# original data
# missing chunking - 3
# not using tokens for search - 2
# too many references to fetch - 1

In [ ]:
top10 = faiss_inference("In what cases should online sexual harassment be criminalized according to the submission?", 10)

In [ ]:
top10[9]

# Better chunking

In [ ]:
# tables = []
# for table in doc.tables():
#     tables.append(table.to_text())

In [ ]:
def flatten_list(nested_list):
    flattened = []
    for item in nested_list:
        if isinstance(item, list):
            flattened.extend(flatten_list(item))
        else:
            flattened.append(item)
    return flattened

In [ ]:
llmsherpa_api_url = "https://readers.llmsherpa.com/api/document/developer/parseDocument?renderFormat=all"
def read_pdf(pdf_url):
    pdf_reader = LayoutPDFReader(llmsherpa_api_url)
    doc = pdf_reader.read_pdf(pdf_url)
    return doc

In [ ]:
def get_section_text(doc, section_title):
    """
    Extracts the text from a specific section in a parsed PDF document.

    Parameters:
    - doc (Document): A Document object from the llmsherpa.readers.layout_reader library.
    - section_title (str): The title of the section to extract.

    Returns:
    - str: The HTML representation of the section's content, or a message if the section is not found.
    """
    selected_section = None
    for section in doc.sections():
        if section.title == section_title:
            selected_section = section
            break

    if not selected_section:
        return f"No section titled '{section_title}' found."
    return selected_section.to_html(include_children=True, recurse=True)

In [ ]:
def rechunker(directory_path):
    all_data = []
    filenames = []
    search_pattern = f"{directory_path}/*.pdf"
    print ()
    for i in glob.glob(search_pattern):
        print (i)
        doc = read_pdf(i)
        sections = []
        # get all the sections in that doc
        for section in doc.sections():
            sections.append(section.title)
            text = get_section_text(doc, section.title)
            all_data.append(text)
            filenames.append(i)
    return all_data, filenames

In [ ]:
def aggregate_string_lengths(lst1,lst2):
    return sum(len(item) for item in lst1 if isinstance(item, str)) + sum(len(item) for item in lst2 if isinstance(item, str))

In [ ]:
def check_criteria(lst):
    if lst[-1] == ":" or "below" in "\n".join(lst):
        return True
    else:
        return False

In [ ]:
def process_doc(doc):
    prev_tag = ""
    doc_sentences = list(doc["sentences"])
    doc_tags = list(doc["tag"])
    merge_list, j = [], -1
    
    for i in range(len(doc)):
        # print (i, prev_tag, doc_tags[i])
        if doc_tags[i] == "para" and prev_tag == "para" and check_criteria(merge_list[j]):
            merge_list[j].extend(doc_sentences[i])
            prev_tag = "para"
            
        elif doc_tags[i] == "list_item" and prev_tag == "para":
            merge_list[j].extend(doc_sentences[i])
            prev_tag = "list_item"
            
        elif doc_tags[i] == "list_item" and prev_tag == "list_item":
            merge_list[j].extend(doc_sentences[i])
            prev_tag = "list_item"
            
        elif doc_tags[i]=="para":
            merge_list.append(doc_sentences[i])
            prev_tag = "para"
            j = j + 1
        else:
            prev_tag = doc_tags[i]
    return merge_list

In [ ]:
def clean_doc(doc):
    doc = [x.encode('ascii', 'ignore').decode('ascii') for x in doc if len(x)>100]
    return doc

In [ ]:
# TAKE: llmsherpa messing up headers but paras are good
# still some paras might be messed up e.g. very small paras that are actually numbers
def get_paras(directory_path):
    all_data = []
    filenames = []
    search_pattern = f"{directory_path}/*.pdf"
    for i in glob.glob(search_pattern):
        print (i)
        doc = read_pdf(i)
        doc = pd.DataFrame(doc.json)
        doc = process_doc(doc)
        doc = ["\n".join(x) for x in doc]
        doc = clean_doc(doc)
        filenames.extend([i]*len(doc))
        all_data.append(doc)
        print (len(all_data), len(filenames))
    return all_data, filenames

In [ ]:
def dedup_list_pairs(list1, list2):
    unique_list1 = set()
    unique_list2 = []
    for i, item in enumerate(list1):
        if item not in unique_list1:
            unique_list1.add(item)
            unique_list2.append(list2[i])
    return list(unique_list1), unique_list2

In [ ]:
all_data_sherpa, filenames_sherpa = get_paras("/Users")

In [ ]:
all_data_sherpa = flatten_list(all_data_sherpa)

In [ ]:
print (len(all_data_sherpa), len(filenames_sherpa))

In [ ]:
all_data_sherpa[63]

In [ ]:
save_path = r"C:\Users\J C SINGLA\Downloads\External - take_home_challenge_(withJSONs)\take_home_challenge_(withJSONs)\data"

In [ ]:
def write_list_to_file(directory, filename, lst):
    if not os.path.exists(directory):
        os.makedirs(directory)
    file_path = os.path.join(directory, filename)
    with open(file_path, 'w') as file:
        json.dump(lst, file)

In [ ]:
def read_list_from_file(directory, filename):
    file_path = os.path.join(directory, filename)
    if os.path.exists(file_path):
        with open(file_path, 'r') as file:
            return json.load(file)
    else:
        return None  # or an empty list, depending on your needs

In [ ]:
all_data_sherpa = read_list_from_file(save_path, "sherpa_paras")

In [ ]:
filenames_sherpa = read_list_from_file(save_path, "sherpa_paras_filenames")

In [ ]:
len(filenames_sherpa)

In [ ]:
write_list_to_file(save_path, "sherpa_paras_filenames", filenames_sherpa)

# Create an index on completed chunks to map to incomplete chunks

In [ ]:
all_data_sherpa

In [ ]:
sentence_embeddings1 = model.encode(all_data_sherpa)

In [ ]:
d = sentence_embeddings.shape[1]
index1 = faiss.IndexFlatL2(d)
index1.add(sentence_embeddings)
index.is_trained

In [ ]:
len(ret_context)

In [ ]:
rechunk_ret_context = []
for i in range(len(ret_context)):
    temp = []
    for j in range(len(ret_context[i])):
        temp.extend(faiss_inference(index1, all_data, ret_context[i][j], 1))
    rechunk_ret_context.append(temp)

In [ ]:
rechunk_ret_context[1][2]

In [ ]:
ret_context[1][2]

In [ ]:
all_data = all_data + all_data_sherpa
filenames = filenames + filenames_sherpa

In [ ]:
# TODO: we need better cleaning on this
all_data_, filenames_ = dedup_list_pairs(all_data, filenames)

In [ ]:
all_data[164]

In [ ]:
# TAKEAWAY: some of these are headers and we want to remove them
# Assumption: The distribution of lengths will be bimodal -> True/bimodal with peak at 25 characters -> let's start removing with 50 characters

In [ ]:
word_counts = [len(sentence) for sentence in all_data]

# Plotting the histogram
plt.figure(figsize=(20, 12))
plt.hist(word_counts, bins=500, alpha=0.7, color='blue', edgecolor='black')
plt.title('Distribution of Sentence Lengths')
plt.xlabel('Number of Words')
plt.ylabel('Frequency')
plt.xticks(range(min(word_counts), max(word_counts) + 1, 25))
plt.xticks(rotation=45)
plt.show()

In [ ]:
# Takeaway: manually inspected the values less than length 100
all_data = [(k,v) for (k, v) in zip(all_data_, filenames_) if len(k)>100]

In [ ]:
filenames = [i[1] for i in all_data]
all_data = [i[0] for i in all_data]

In [ ]:
sentence_embeddings = model.encode(all_data)

In [ ]:
words = """
The Student Liaison Committee had another successful year in attracting schools from the Pacific 
Northwest to compete in the Oregon Section's annual Traffic Bowl Competition. The Traffic Bowl is 
a Jeopardy-based trivia contest where students must answer questions on a variety of traffic and 
transportation trivia. This year the competition was held on November 15, 2007 at McMenamin's 
Edgefield just east of Portland, Oregon.  
This year we had 52 students from six universities attending the event. Participating schools were: 
Portland State University 
University of Idaho 
University of Portland 
Oregon State University 
University of Washington 
Oregon Institute of Technology 
"""

In [ ]:
len(words)